In [16]:
import pandas as pd
import datetime
import yfinance as yf
import backtrader as bt
import numpy as np
import warnings

In [17]:
from Data_Preprocess import IAM_data_setup, download_IAM, SelectIndex, generate_interval
from Data_Preprocess import calculate_returns, calculate_date_range

In [18]:
assets, asset_classes = IAM_data_setup()

In [19]:
display(asset_classes['Sector'].unique())
display(asset_classes['Class'].unique())

array(['Consumer Defensive', 'Real Estate', 'Industrials', 'Technology',
       'Cash', 'Municipal', 'Financial Services', 'Energy', 'Index',
       'Corporate', 'Healthcare', 'Consumer Cyclical', 'International',
       'Treasury', 'Ultrashort', 'Basic Materials', 'High Yield', 'ETF',
       'Agency'], dtype=object)

array(['Equity', 'Cash', 'Fixed Income', 'Commodity', 'Crypto'],
      dtype=object)

In [20]:
print(assets)

['0341.HK', '0405.HK', '0737.HK', '0P00001DU6', '0P00002C5C', '0P00006OXK.T', '0P0000UIC0', '0P0000VP5F', '0P0000YVJG', '0P0000Z5AZ', '0P0000ZJFK', '0P00013N6C', '0P0001DFIZ', '0P0001DHN3', '0P0001Q45W.HK', '0P0001Q87U', '1186.HK', '1810.HK', '3175.HK', '3187.HK', '9988.HK', 'ADDYY', 'AGGU.L', 'ARKG', 'ARKK', 'BABA', 'CSBGU7.SW', 'ERND.L', 'GLD', 'GOVT', 'HYGH', 'HYLD.L', 'HYMB', 'IAGG', 'IBIT', 'IBTS.SW', 'IDEV', 'IDTL.L', 'IDTM.L', 'IEMB.L', 'IGIL.L', 'IGSB', 'IHYU.L', 'ISAG.L', 'ISTB', 'IVE', 'IWRD.L', 'JDN.MX', 'LQDE.L', 'METV', 'PAVE', 'SDHY.L', 'SHV', 'TFLO', 'USO', 'ZM']


In [21]:
display(asset_classes.shape)
display(asset_classes.T)

(56, 6)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
Assets,0341.HK,0405.HK,0737.HK,0P00001DU6,0P00002C5C,0P00006OXK.T,0P0000UIC0,0P0000VP5F,0P0000YVJG,0P0000Z5AZ,...,JDN.MX,LQDE.L,METV,PAVE,SDHY.L,SHV,TFLO,USO,ZM,0P0001Q45W.HK
Class,Equity,Equity,Equity,Equity,Cash,Equity,Fixed Income,Fixed Income,Fixed Income,Equity,...,Equity,Fixed Income,Equity,Equity,Fixed Income,Fixed Income,Fixed Income,Commodity,Equity,Cash
Sector,Consumer Defensive,Real Estate,Industrials,Technology,Cash,Industrials,Municipal,Financial Services,Energy,Index,...,Consumer Defensive,Corporate,Technology,Industrials,High Yield,Treasury,Treasury,Energy,Technology,Cash
Type,Stock,RIT,Stock,Fund,Fund,Fund,Fund,Fund,Fund,Fund,...,Stock,ETF,ETF,ETF,ETF,ETF,ETF,ETF,Stock,Fund
Currency,HKD,HKD,HKD,USD,USD,JPY,USD,USD,USD,USD,...,MXN,USD,USD,USD,USD,USD,USD,USD,USD,HKD
Rate,0.128205,0.128205,0.128205,1.0,1.0,0.0066,1.0,1.0,1.0,1.0,...,0.05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.128205


In [22]:
asset_classes[asset_classes['Assets'] == '0P0001Q45W.HK']

,Assets,Class,Sector,Type,Currency,Rate
55,0P0001Q45W.HK,Cash,Cash,Fund,HKD,0.128205


In [24]:
# Download and generate full data file from yfinance
df = yf.download(assets, start=start, end=end)

[*********************100%***********************]  56 of 56 completed


In [26]:
df.to_csv("IAM_full_data.txt")

In [23]:
# Downloading data
data = download_IAM()
start = data.index[0]
end = data.index[-1]
print(start, " , ", end)
display(data.head())
display(data.tail())

2013-01-02 00:00:00+00:00  ,  2024-09-27 00:00:00+00:00


Price                     Adj Close                                        \
Ticker                      0341.HK 0405.HK 0737.HK 0P00001DU6 0P00002C5C   
Date                                                                        
2013-01-02 00:00:00+00:00   15.0428  1.6463  1.8862        NaN        NaN   
2013-01-03 00:00:00+00:00   15.1463  1.6507  1.8949        NaN        NaN   
2013-01-04 00:00:00+00:00   15.1118  1.6507  1.8906        NaN        NaN   
2013-01-07 00:00:00+00:00   15.1118  1.6906  1.9123        NaN        NaN   
2013-01-08 00:00:00+00:00   15.2153  1.6817  1.9036        NaN        NaN   

Price                                                                    \
Ticker                    0P00006OXK.T 0P0000UIC0 0P0000VP5F 0P0000YVJG   
Date                                                                      
2013-01-02 00:00:00+00:00          NaN        NaN        NaN        NaN   
2013-01-03 00:00:00+00:00          NaN        NaN        NaN        NaN   
2013-01-04 00:00:00+00:00          NaN        NaN        NaN        NaN   
2013-01-07 00:00:00+00:00          NaN        NaN        NaN        NaN   
2013-01-08 00:00:00+00:00          NaN        NaN        NaN        NaN   

Price                                 ...     Volume                        \
Ticker                    0P0000Z5AZ  ...     IWRD.L JDN.MX    LQDE.L METV   
Date                                  ...                                    
2013-01-02 00:00:00+00:00        NaN  ...   502324.0    NaN   22658.0  NaN   
2013-01-03 00:00:00+00:00        NaN  ...   190327.0    NaN   71159.0  NaN   
2013-01-04 00:00:00+00:00        NaN  ...  1066404.0    NaN   80771.0  NaN   
2013-01-07 00:00:00+00:00        NaN  ...   716990.0    NaN   54127.0  NaN   
2013-01-08 00:00:00+00:00        NaN  ...   712517.0    NaN  141965.0  NaN   

Price                                                               
Ticker                    PAVE SDHY.L       SHV TFLO       USO  ZM  
Date                                                                
2013-01-02 00:00:00+00:00  NaN    NaN  259200.0  NaN  693100.0 NaN  
2013-01-03 00:00:00+00:00  NaN    NaN  357500.0  NaN  489425.0 NaN  
2013-01-04 00:00:00+00:00  NaN    NaN  303100.0  NaN  742613.0 NaN  
2013-01-07 00:00:00+00:00  NaN    NaN  288400.0  NaN  356675.0 NaN  
2013-01-08 00:00:00+00:00  NaN    NaN  140900.0  NaN  480188.0 NaN  

[5 rows x 330 columns]

Price                     Adj Close                                        \
Ticker                      0341.HK 0405.HK 0737.HK 0P00001DU6 0P00002C5C   
Date                                                                        
2024-09-23 00:00:00+00:00      7.73    0.95    1.86      83.57     1.9333   
2024-09-24 00:00:00+00:00      7.76    0.98    1.86      85.92     1.9336   
2024-09-25 00:00:00+00:00      7.87    1.01    1.83      86.21     1.9339   
2024-09-26 00:00:00+00:00      8.04    1.03    1.85      89.35     1.9341   
2024-09-27 00:00:00+00:00      8.36    1.11    1.85      91.95     1.9344   

Price                                                                    \
Ticker                    0P00006OXK.T 0P0000UIC0 0P0000VP5F 0P0000YVJG   
Date                                                                      
2024-09-23 00:00:00+00:00       2895.0    19.1797      18.19      15.33   
2024-09-24 00:00:00+00:00       2876.0    19.1797      18.20      15.33   
2024-09-25 00:00:00+00:00       2862.0    19.1797      18.19      15.33   
2024-09-26 00:00:00+00:00       2945.0    19.1797      18.20      15.33   
2024-09-27 00:00:00+00:00       2860.0    19.1797      18.22      15.33   

Price                                 ...   Volume                    \
Ticker                    0P0000Z5AZ  ...   IWRD.L  JDN.MX    LQDE.L   
Date                                  ...                              
2024-09-23 00:00:00+00:00     289.84  ...  15022.0   667.0   20396.0   
2024-09-24 00:00:00+00:00     290.82  ...  13535.0  3002.0  272396.0   
2024-09-25 00:00:00+00:00     291.34  ...  19582.0  1316.0  128626.0   
2024-09-26 00:00:00+00:00     294.07  ...  26075.0  4964.0  509089.0   
2024-09-27 00:00:00+00:00     292.70  ...  34924.0  1560.0   83029.0   

Price                                                                          \
Ticker                         METV       PAVE   SDHY.L        SHV       TFLO   
Date                                                                            
2024-09-23 00:00:00+00:00  215800.0   628600.0   1364.0  3188200.0  1797400.0   
2024-09-24 00:00:00+00:00  143000.0   586100.0   1697.0  3068800.0  1876900.0   
2024-09-25 00:00:00+00:00  260900.0  1364500.0  10793.0  3090200.0   975800.0   
2024-09-26 00:00:00+00:00  231200.0   699300.0  32704.0  3071100.0  1301200.0   
2024-09-27 00:00:00+00:00  177500.0   939400.0   3897.0  4247700.0  2162400.0   

Price                                            
Ticker                           USO         ZM  
Date                                             
2024-09-23 00:00:00+00:00  2837700.0  2526600.0  
2024-09-24 00:00:00+00:00  2387000.0  2294700.0  
2024-09-25 00:00:00+00:00  3469200.0  2479900.0  
2024-09-26 00:00:00+00:00  4593800.0  2018900.0  
2024-09-27 00:00:00+00:00  3214700.0  1417500.0  

[5 rows x 330 columns]

In [ ]:
#  all test is from the 1004th day and finish at the last testdata day.
#
start_test = 1004
end_test = data.shape[0] - 1
test_size = 1000
print(f" Testing data from {start_test} to {end_test}")

In [ ]:
rebalance_index = {}
rebalance_index["M"] = SelectIndex(data, mode="M", start_index=start_test)
rebalance_index["Q"] = SelectIndex(data, mode="Q", start_index=start_test)
rebalance_index["S"] = SelectIndex(data, mode="S", start_index=start_test)
print(rebalance_index)

In [ ]:
r_index={}
r_index["M"] = generate_interval(data, rebalance_index["M"], "rebalance_index_M.csv")
r_index["Q"] = generate_interval(data, rebalance_index["Q"], "rebalance_index_Q.csv")
r_index["S"] = generate_interval(data, rebalance_index["S"], "rebalance_index_S.csv")
r_index["S"]

In [ ]:
returns = calculate_returns(data)
n=display(returns)

In [ ]:
df = calculate_date_range(data, returns.columns.to_list())
display(df)


In [ ]:
print(returns.shape)
print(data.shape)

In [ ]:
data

In [ ]:
display(data.index[0], data.index[-1])
display(data.head())